In [1]:
import time
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error

import rnn
import ltc
import viz

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [2]:

train_data = pd.read_csv('DailyDelhiClimateTrain.csv')
train_data['date'] = pd.to_datetime(train_data['date'])
train_data.set_index('date', inplace=True)
features = train_data[['meantemp', 'humidity']]
scaler = MinMaxScaler(feature_range=(0, 1))
features_scaled = scaler.fit_transform(features)
X, y = rnn.create_sequences(features_scaled, seq_length=30)
split_ratio = 0.8
split = int(len(X) * split_ratio)
X_train, X_val = X[:split], X[split:]
y_train, y_val = y[:split], y[split:]
X_train, y_train = torch.Tensor(X_train).to(device), torch.Tensor(y_train).to(device)
X_val, y_val = torch.Tensor(X_val).to(device), torch.Tensor(y_val).to(device)
train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=16, shuffle=True)
val_loader = DataLoader(TensorDataset(X_val, y_val), batch_size=16)


test_data = pd.read_csv('DailyDelhiClimateTest.csv')
test_data['date'] = pd.to_datetime(test_data['date'])
test_data.set_index('date', inplace=True)
test_features = test_data[['meantemp', 'humidity']]
test_features_scaled = scaler.transform(test_features)
X_test, y_test = rnn.create_sequences(test_features_scaled, seq_length=30)
X_test, y_test = torch.Tensor(X_test).to(device), torch.Tensor(y_test).to(device)
test_loader = DataLoader(TensorDataset(X_test, y_test), batch_size=16)

In [3]:
results_dict = {}
model_name = []

In [4]:
ltc_1 = ltc.ParallelMultiScaleLTCModel(input_size=2, hidden_size=50, output_size=2,tau1=0.7, tau2=1.0, tau3=10.0, tau4=0.7).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(ltc_1.parameters(), lr=0.0001)

ltc_1,ltc1_time,epoch_dx_dt_history = ltc.train_LTC_model(train_loader, val_loader, ltc_1, criterion, optimizer, device)


predictions, actuals, mse_temp, mse_humidity, mase_temp, mase_humidity, allocated_memory, reserved_memory = ltc.evaluate_LTC_model(test_loader, ltc_1, device)
results = {'train_time': ltc1_time, 'allocated_memory': allocated_memory, 'reserved_memory': reserved_memory, 'epoch_dx_dt_history':epoch_dx_dt_history,
 'mse_temp':mse_temp,'mse_humidity':mse_humidity, 'mase_temp':mase_temp, 'mase_humidity':mase_humidity}
results_dict[model_name[1]] = results

viz.plot_results(predictions, actuals)

viz.plot_dx_dt_history(epoch_dx_dt_history)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (16x2 and 50x50)